# Visualización de Datos Climáticos con Python

## Introducción

Hoy vamos a trabajar con datos climáticos usando una API pública y visualizarlos en un mapa interactivo. Este ejercicio es ideal para aprender a combinar la recolección de datos de internet con herramientas de visualización geoespacial.

## Objetivos de la clase:

Usar la API de OpenWeatherMap para obtener datos climáticos.
Crear mapas interactivos con Folium.
Personalizar los mapas con colores, iconos y capas.

## Paso 1: Importar librerías necesarias
Primero, vamos a cargar las herramientas que usaremos:

In [1]:
import requests
import folium

* ```requests```: Es una librería que usamos para conectarnos a internet y descargar información de una API.
* ```folium```: Es la estrella del día; con esta herramienta crearemos mapas interactivos.

Siempre que empieces un proyecto en Python, importa solo las librerías que realmente necesites para mantener tu código organizado.

## Paso 2: Conectar con la API de OpenWeatherMap

Ahora necesitamos conectarnos a la API para obtener datos climáticos. Para esto, necesitamos una **clave de acceso (API key)**, que se obtiene al registrarse en [OpenWeatherMap](https://openweathermap.org/).

Aquí configuramos los parámetros básicos:

In [2]:
# Parámetros para la API
api_key = "ad6e06e230f7307598923265cf881c17"
cities = ["Bogotá", "Medellín", "Cali", "Barranquilla"]  # Ciudades objetivo
base_url = "http://api.openweathermap.org/data/2.5/weather"

1. **api_key:** Es tu identificador único para usar la API.
2. **cities:** Es una lista de las ciudades de las que queremos obtener información.
3. **base_url:** Es la URL de la API para obtener datos climáticos actuales.

Ejemplo de solicitud para Bogotá:

In [3]:
# Obtener datos para una ciudad
params = {"q": "Bogotá", "appid": api_key, "units": "metric"}
response = requests.get(base_url, params=params)
data = response.json()

print(data)  # Explorar los datos obtenidos

{'coord': {'lon': -74.0817, 'lat': 4.6097}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 10.81, 'feels_like': 10.56, 'temp_min': 10.81, 'temp_max': 11.49, 'pressure': 1018, 'humidity': 100, 'sea_level': 1018, 'grnd_level': 737}, 'visibility': 10000, 'wind': {'speed': 0.6, 'deg': 76, 'gust': 1.04}, 'clouds': {'all': 59}, 'dt': 1733207254, 'sys': {'type': 2, 'id': 2099091, 'country': 'CO', 'sunrise': 1733223028, 'sunset': 1733265756}, 'timezone': -18000, 'id': 3688689, 'name': 'Bogota', 'cod': 200}


* **q:** Es el nombre de la ciudad.
* **appid:** Es tu clave de acceso.
* **units:** Especifica el sistema métrico para la temperatura (°C).

**Resultado:** El servidor responde con un diccionario que contiene información como temperatura, humedad y descripción del clima.

## Paso 3: Crear una función para obtener datos de cualquier ciudad

Escribir funciones nos permite evitar repetir código. Aquí está nuestra función:

In [4]:
def get_city_weather(city, api_key):
    params = {"q": city, "appid": api_key, "units": "metric"}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return {
            "city": data["name"],
            "lat": data["coord"]["lat"],
            "lon": data["coord"]["lon"],
            "temp": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "weather": data["weather"][0]["description"].capitalize(),
            "wind_speed": data["wind"]["speed"],
            "icon": data["weather"][0]["icon"]
        }
    else:
        print(f"Error al obtener datos para {city}")
        return None

* **Entrada:** Nombre de la ciudad y tu clave de API.
* **Salida:** Un diccionario con información relevante: nombre, coordenadas, temperatura, humedad, clima, velocidad del viento e ícono.

Esto hace que sea fácil procesar varias ciudades.

## Paso 4: Obtener datos para múltiples ciudades

Ahora, aplicamos nuestra función a todas las ciudades de la lista.

In [5]:
weather_data = [get_city_weather(city, api_key) for city in cities if get_city_weather(city, api_key)]
print(weather_data)

[{'city': 'Bogota', 'lat': 4.6097, 'lon': -74.0817, 'temp': 10.81, 'humidity': 100, 'weather': 'Broken clouds', 'wind_speed': 0.6, 'icon': '04n'}, {'city': 'Medellín', 'lat': 6.2518, 'lon': -75.5636, 'temp': 17.79, 'humidity': 96, 'weather': 'Overcast clouds', 'wind_speed': 8.94, 'icon': '04n'}, {'city': 'Santiago de Cali', 'lat': 3.4372, 'lon': -76.5225, 'temp': 18.31, 'humidity': 88, 'weather': 'Overcast clouds', 'wind_speed': 2.24, 'icon': '04n'}, {'city': 'Barranquilla', 'lat': 10.9639, 'lon': -74.7964, 'temp': 26.31, 'humidity': 85, 'weather': 'Overcast clouds', 'wind_speed': 2.7, 'icon': '04n'}]


Aquí usamos una lista por comprensión:

* Llama a la función get_city_weather para cada ciudad.
* Filtra las ciudades para las que no se pudo obtener información.

El resultado es una lista de diccionarios con datos climáticos.

## Paso 5: Crear un mapa base

Para visualizar los datos, primero creamos un mapa centrado en Colombia.

In [6]:
# Coordenadas aproximadas de Colombia
center_coords = [4.570868, -74.297333]
map_base = folium.Map(location=center_coords, zoom_start=6)

1. **location:** Coordenadas iniciales del mapa.
2. **zoom_start:** Nivel de zoom inicial.

## Paso 6: Añadir marcadores al mapa

Ahora, colocamos un marcador en el mapa para cada ciudad.

In [7]:
for city in weather_data:
    folium.Marker(
        location=[city["lat"], city["lon"]],
        popup=f"{city['city']}: {city['temp']} °C",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map_base)

* **location:** Ubicación geográfica de la ciudad.
* **popup:** Texto que aparece al hacer clic en el marcador.
* **icon:** Cambia el estilo del marcador.

## Paso 7: Guardar el mapa
El mapa se guarda en un archivo HTML para que puedas abrirlo en cualquier navegador.

In [8]:
map_base.save("mapa_clima_inicial.html")

## Paso 8: Personalizar el mapa
Podemos hacer que el mapa sea más interactivo y visual:

1. **Colores dinámicos según la temperatura:**

In [9]:
def get_color(temp):
    if temp < 10:
        return "blue"
    elif 10 <= temp <= 20:
        return "green"
    elif 20 < temp <= 30:
        return "orange"
    else:
        return "red"

Esto asigna un color a cada marcador según la temperatura.

2. **Añadir círculos escalados:**

In [10]:
for city in weather_data:
    folium.CircleMarker(
        location=[city["lat"], city["lon"]],
        radius=city["temp"] / 2,  # Escalar el tamaño del círculo con la temperatura
        color=get_color(city["temp"]),
        fill=True,
        fill_opacity=0.6,
        popup=f"Temperatura: {city['temp']} °C"
    ).add_to(map_base)

map_base.save("mapa_clima_final.html")

Los círculos muestran visualmente la temperatura de cada ciudad.

## Conclusión
Con este proyecto, aprendimos a:

1. Conectarnos a una API para obtener datos en tiempo real.
2. Procesar datos geoespaciales.
3. Crear mapas interactivos personalizados.

El resultado final es un mapa atractivo y funcional que combina datos reales con visualización interactiva.